In [23]:
#install pyDF2
#!pip install PyPDF2

#this is also fitz
#!pip install PyMuPDF

In [ ]:
# importing all the required modules
#import PyPDF2 replaces with PyMuPDF
import fitz
import json
import dateutil.parser
import datetime
import re

In [33]:
#Get all text from pdf
doc = fitz.open('test.pdf')  # open document
text = ''
for page in doc:  # iterate the document pages
    text += page.get_text() + "\n"
doc.close()

#print(text)

In [7]:
#Returns the textsize and font of every line in a pdf.
def getTextFormat(pdf):
    results = []
    for page in pdf:
            dict = page.get_text("dict")
            blocks = dict["blocks"]
            for block in blocks:
                if "lines" in block.keys():
                    spans = block['lines']
                    for span in spans:
                        data = span['spans']
                        for lines in data:
                                results.append((lines['text'], lines['size'], lines['font']))
    return results


In [8]:
def GetSection(pdf, start, stop, fontSize, fonts):
    section = findBetween(start,stop,pdf)
    clean = filterFont(fontSize,fonts,section) 

    text = ""
    for line in clean:
        text += line + "\n"
        continue
        
    return text

In [9]:
def filterFont(fontSize, fontList, section):
    res = []
    for (txt, size, font) in section:
        #remove empty txt or only spaces
        if txt == " " or txt.count(' ') == len(txt):
            continue
        
        if round(size,1) in fontSize or fontSize == []:
            if font in fontList or fontList == []:
                res.append(txt)
    return res

In [10]:
#returns all lines from and including start till stop for a given pdf

def findBetween (start, stop, pdf):
    results = []
    save = False
    for page in pdf:
            dict = page.get_text("dict")
            blocks = dict["blocks"]
            for block in blocks:
                if "lines" in block.keys():
                    spans = block['lines']
                    for span in spans:
                        data = span['spans']
                        for lines in data:
                            if start in lines['text']: 
                                save = True
                            if save:    
                                results.append((lines['text'], lines['size'], lines['font']))
                            if stop in lines['text']and save:
                                return results
    return results

In [11]:
#nog kijken welke handiger is boven of deze.
#returns all lines from and including start till stop for a given format

def findBetweenFormat (start, stop, form, inclusive):
    results = []
    save = False
    index = 0
    for (txt, size, font) in form:
        if start in txt or start == '': 
            save = True
        if stop in txt and save:
            if inclusive:
                #includes stop as tail, remainder is without stop
                results.append((txt, size, font)) 
            else:
                index -= 1   #returns stop as header for remainder
            return (results,form[index+1:])
        if save:    
            results.append((txt, size, font))
        index += 1
    return (results,[])

In [12]:
#Scans and ordens EC orphan designation pdfs
def getTableEC(pdf):
    table = {'intro': dict.fromkeys(['front_page']),
             'body' : dict.fromkeys(['main_text','P1','P2','P3']),
             'decision' : dict.fromkeys(['dec_text','A1','A2','A3']) }

    pdf_format = getTextFormat(pdf)

    #intro
    (content,rem) = findBetweenFormat('DECISION','AUTHENTIC',pdf_format,True)
    table['intro']['front_page'] = filterFont([],[],content)

    #filter second intro message
    (_,rem) = findBetweenFormat('DECISION','AUTHENTIC',rem,True)

    #content
    (content,rem) = findBetweenFormat('COMMISSION','Whereas:',rem,True)
    table['body']['main_text'] = filterFont([],[],content)

    (content,rem) = findBetweenFormat('(1)','(2)',rem,False)
    table['body']['P1'] = filterFont([],[],content)

    (content,rem) = findBetweenFormat('(2)','(3)',rem,False)
    table['body']['P2'] = filterFont([],[],content)

    (content,rem) = findBetweenFormat('(3)','HAS ADOPTED THIS DECISION:',rem,False)
    table['body']['P3'] = filterFont([],[],content)

    #decision
    (content,rem) = findBetweenFormat('HAS ADOPTED THIS DECISION:','Article 1',rem,False)
    table['decision']['dec_text'] = filterFont([],[],content)

    (content,rem) = findBetweenFormat('Article 1','Article 2',rem,False)
    table['decision']['A1'] = filterFont([12],[],content)

    (content,rem) = findBetweenFormat('Article 2','Article 3',rem,False)
    table['decision']['A2'] = filterFont([12],[],content)

    (content,rem) = findBetweenFormat('Article 3','Done at',rem,False)
    table['decision']['A3'] = filterFont([12],[],content)
    
    return table


In [60]:
def tableEC_getDate(table):
    section = table['intro']['front_page']
    for txt in section:
        if 'of ' in txt:
            return getDate(txt)
    return getDate('')

In [84]:
def tableEC_getName(table):
    section = table['intro']['front_page']
    for txt in section:
        if '\"' in txt:
            return(re.search('"([^"]*)"',txt)[0])
    return "Name Not Found"

In [58]:
def getDate(txt):
    try:
        return dateutil.parser.parse(txt, fuzzy = True)
    except:
        return datetime.datetime(1980, 1, 1, 0, 0)

In [91]:

pdf = fitz.open('test_xenon.pdf')  # open document

table = getTableEC(pdf)  #get formated dictionary

pdf.close() #close document

print(tableEC_getDate(table))
print(tableEC_getName(table))
print(json.dumps(table, indent=3))

2015-04-24 00:00:00
"Xenon"
{
   "intro": {
      "front_page": [
         "COMMISSION IMPLEMENTING DECISION ",
         "of 24.4.2015 ",
         "relating to the designation of \"Xenon\" as an orphan medicinal product under ",
         "Regulation (EC) No 141/2000 of the European Parliament and of the Council ",
         "(Text with EEA relevance) ",
         "(ONLY THE ENGLISH TEXT IS AUTHENTIC) "
      ]
   },
   "body": {
      "main_text": [
         "THE EUROPEAN COMMISSION, ",
         "Having regard to the Treaty on the Functioning of the European Union, ",
         "Having regard to Regulation (EC) No 141/2000 of the European Parliament and of the Council ",
         "of 16 December 1999 on orphan medicinal products",
         "1",
         ", and in particular the first sentence of ",
         "Article 5(8) thereof, ",
         "Having regard to the application submitted by Neuroprotexeon Ltd on 9 December 2014 under ",
         "Article 5(1) of Regulation (EC) No 141/2000, 